In [1]:
import os
USE_HPC=False
if USE_HPC:
    mycache_dir="/data/cmpe249-fa23/Huggingfacecache"
    os.environ['TRANSFORMERS_CACHE'] = mycache_dir
    os.environ['HF_HOME'] = mycache_dir
    os.environ['HF_DATASETS_CACHE'] = mycache_dir
    os.environ['http_proxy'] = "http://172.16.1.2:3128"
    os.environ['HTTP_PROXY'] = "http://172.16.1.2:3128"
    os.environ['https_proxy'] = "https://172.16.1.2:3128"
    os.environ['HTTPS_PROXY'] = "https://172.16.1.2:3128"
    trainoutput="/data/cmpe249-fa23/trainoutput/huggingface"
    taskname="Gastroasksciencemodeling"
else:
    trainoutput="./output"
    taskname="Gastroasksciencemodeling"

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

# Load the train split and limit to the first 5000 examples
gastro = load_dataset("gayanin/pubmed-gastro-maskfilling", split="train[:2000]")

# Display a sample
print(gastro)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


masked_pubmed_highlights_dataset.csv:   0%|          | 0.00/2.50M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11772 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'highlights', 'masked_highlights'],
    num_rows: 2000
})


In [4]:
gastro = gastro.train_test_split(test_size=0.2)

In [5]:
gastro["train"][0]

{'id': 10929900,
 'highlights': 'Prevalence and clinical features of heterotopic gastric mucosa in the upper oesophagus (inlet patch).',
 'masked_highlights': '<mask> and clinical features of <mask> <mask> <mask> in the upper oesophagus (inlet patch).'}

In [6]:
#extract the text subfield from its nested structure with the flatten method:
gastro = gastro.flatten()
gastro

DatasetDict({
    train: Dataset({
        features: ['id', 'highlights', 'masked_highlights'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['id', 'highlights', 'masked_highlights'],
        num_rows: 400
    })
})

In [7]:
!pip install transformers

In [8]:
from transformers import AutoTokenizer
modelname="albert-base-v2"
if USE_HPC:
    localpath=os.path.join(mycache_dir, modelname)
    tokenizer = AutoTokenizer.from_pretrained(localpath)
else:
    tokenizer = AutoTokenizer.from_pretrained(modelname)#, cache_dir=mycache_dir)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [9]:
tokenizer(gastro["train"][0]['masked_highlights'])

{'input_ids': [2, 13, 1, 23265, 1, 17, 5611, 967, 16, 13, 1, 23265, 1, 13, 1, 23265, 1, 13, 1, 23265, 1, 19, 14, 1508, 635, 160, 9976, 14972, 18, 13, 5, 108, 1336, 7331, 6, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
examples=gastro["train"]

In [11]:
len(examples)

1600

In [12]:
listexamples = [" ".join(x) for x in examples["highlights"]]

In [13]:
token_train=tokenizer(listexamples)

In [14]:
token_train=tokenizer(listexamples, padding='max_length', truncation=True)

In [15]:
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def tokenize_function(self, examples):
        return self.tokenizer(
            [" ".join(x) for x in examples["masked_highlights"]],
            padding="max_length",
            truncation=True,
        )

In [16]:
tokenizer_wrapper = TokenizerWrapper(tokenizer)

In [17]:
tokenized_dataset = gastro.map(tokenizer_wrapper.tokenize_function, batched=True, num_proc=3, remove_columns=gastro["train"].column_names)

Map (num_proc=3):   0%|          | 0/1600 [00:00<?, ? examples/s]

Map (num_proc=3):   0%|          | 0/400 [00:00<?, ? examples/s]

In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [19]:
def group_texts(examples):
    block_size = 128
    # Concatenate all texts.
    #print(examples.keys())
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    #print('total_length:', total_length)
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [20]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/1600 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/400 [00:00<?, ? examples/s]

In [21]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6400
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1600
    })
})

In [22]:
from transformers import DataCollatorForLanguageModeling

#tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [23]:
from transformers import AutoModelForMaskedLM
if USE_HPC:
    localpath=os.path.join(mycache_dir, modelname) #modelname="distilroberta-base"
    model = AutoModelForMaskedLM.from_pretrained(localpath)
else:
    model = AutoModelForMaskedLM.from_pretrained(modelname)#"distilroberta-base")

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForMaskedLM: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
text = f"The cat sat on the {tokenizer.mask_token} and looked out the window."

In [25]:
inputs = tokenizer(text, return_tensors="pt")

In [26]:
import torch
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_index

tensor([6])

In [27]:
logits = model(**inputs).logits

In [28]:
logits

tensor([[[  3.9786,   2.2369, -12.7231,  ...,  -5.9933, -10.3969,  -7.1802],
         [  4.4506,  -0.6332,  -9.2162,  ...,  -8.0639, -19.4058,  -6.9170],
         [ -1.1265,  -2.7565,  -5.0089,  ...,  -7.9603,  -7.9703,  -6.2093],
         ...,
         [ -6.1774,  -3.9787,  -0.7075,  ...,  -4.5568,  -5.2074,  -6.1452],
         [ -0.9738,   2.8613,   0.2254,  ...,  -0.5538,  -3.4019,   1.7377],
         [  0.6026,   3.3788,  -6.1933,  ...,  -4.8841,  -3.3917,  -3.7171]]],
       grad_fn=<ViewBackward0>)

In [29]:
logits.shape

torch.Size([1, 14, 30000])

In [30]:
mask_token_logits = logits[0, mask_token_index, :]
mask_token_logits

tensor([[-5.7512, -9.3671, -2.5039,  ..., -9.6285, -4.5236, -4.9660]],
       grad_fn=<IndexBackward0>)

In [31]:
mask_token_logits.shape

torch.Size([1, 30000])

In [32]:
top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

In [33]:
top_3_tokens

[4914, 10795, 803]

In [34]:
for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

The cat sat on the couch and looked out the window.
The cat sat on the sofa and looked out the window.
The cat sat on the floor and looked out the window.


In [35]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import math

def compute_perplexity(model, tokenizer, text):
    """
    Compute perplexity for a text using a masked language model.
    """
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    # Mask a random token (excluding special tokens)
    mask_token_index = torch.where(input_ids[0] == tokenizer.mask_token_id)[0]
    if len(mask_token_index) == 0:
        raise ValueError("Text does not contain a mask token. Ensure you include '<mask>' in the input.")

    # Get model outputs
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Calculate probabilities for the masked token
    log_probs = 0
    for idx in mask_token_index:
        token_logits = logits[0, idx]
        target_token_id = input_ids[0, idx]
        token_prob = torch.softmax(token_logits, dim=0)[target_token_id]
        log_probs += torch.log(token_prob)

    # Average the log probabilities
    avg_log_prob = log_probs / len(mask_token_index)

    # Compute perplexity
    perplexity = torch.exp(-avg_log_prob)
    return perplexity.item()


perplexity = compute_perplexity(model, tokenizer, text)
print(f"Perplexity: {perplexity}")

Perplexity: 26829344768.0


In [36]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=os.path.join(trainoutput, modelname, taskname), #"./output/my_awesome_eli5_mlm_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [37]:
from huggingface_hub import notebook_login
notebook_login()

In [38]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

In [39]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.445800,1.019455
2,0.964300,0.866164
3,0.892300,0.818745
4,0.837800,0.796437
5,0.766600,0.756450
6,0.742600,0.715671
7,0.703000,0.680704
8,0.679200,0.673991
9,0.648600,0.637382
10,0.617600,0.652733


TrainOutput(global_step=8000, training_loss=0.8136476554870605, metrics={'train_runtime': 1758.649, 'train_samples_per_second': 36.392, 'train_steps_per_second': 4.549, 'total_flos': 359590526976000.0, 'train_loss': 0.8136476554870605, 'epoch': 10.0})

In [40]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 1.90


In [41]:
import pickle

# Save the model
with open("trained_model_v2.pkl", "wb") as f:
    pickle.dump(model, f)

In [42]:
with open('trained_model_v2.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [43]:
loaded_model

AlbertForMaskedLM(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True

In [44]:
import zipfile
import os

# Define the folder or files you want to zip
folder_to_zip = "/content/output"
zip_file_name = "model_v2_output.zip"

# Create a zip file
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for root, dirs, files in os.walk(folder_to_zip):
        for file in files:
            file_path = os.path.join(root, file)
            # Add file to the zip archive
            zipf.write(file_path, os.path.relpath(file_path, folder_to_zip))

print(f"Zipped folder into {zip_file_name}")

Zipped folder into model_v2_output.zip
